In [1]:
"""
import shutil
shutil.copy("./drive/MyDrive/2048.zip", './')
shutil.copy("./drive/MyDrive/custom_layers2.py", './')
shutil.unpack_archive("./2048.zip", "./")
"""

'\nimport shutil\nshutil.copy("./drive/MyDrive/2048.zip", \'./\')\nshutil.copy("./drive/MyDrive/custom_layers2.py", \'./\')\nshutil.unpack_archive("./2048.zip", "./")\n'

In [1]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow import keras
import matplotlib
import matplotlib.pyplot as plt
import opt_calc as cl
import opt_module as lens
import opt_util

In [14]:
#モード選択
mode = "image"

In [15]:
#訓練データ用
#数字あり

if mode=="point":
    count = 1 #繰り返す回数,ファイルの数(つまりデータの総数はdata_train*count)
    data_train = 1 #1ファイルあたりの元画像の枚数
    batch_size = 1

elif mode=="image":
    count = 1 #繰り返す回数,ファイルの数(つまりデータの総数はdata_train*count)
    data_train = 8 #1ファイルあたりの元画像の枚数
    batch_size = 8
    
elif mode=="test":
    count = 1 #繰り返す回数,ファイルの数(つまりデータの総数はdata_train*count)
    data_train = 96 #1ファイルあたりの枚数
    batch_size = 2
    test_bias = 99999


#分割前の画像
height = 1024
width = 1024

#分割後のデータ
height_s = 256
width_s = 256

#学習・検証データ用分割数
sh_num=8
sw_num=8

#テストデータ用分割数
img_height_step = 15
img_width_step = 15

#伝搬計算用
pitch= 6.4*pow(10.0, -6.0)  #サンプリングピッチ
Lambda = 532.0 * pow(10.0, -9.0)  #波長
z1 = 0.2
z2 = 0.1
z3 = 0.2
fti = 10

In [16]:
if mode=="point":
    batch_size = 1
    #1
    x_train=np.ones((1,1024,1024)).astype(np.float32)
    x_train=x_train.astype(np.float32)

    #物体点情報の作成
    X_train = np.zeros((batch_size,height,width,2), dtype='float32')
    for i in range(batch_size):
        X_train[i,0:height,0:width,0]=x_train[i,:,:]
    X_train1 =X_train

    #2
    #回折計算
    result1 = cl.angular_spectrum(X_train1, wl=Lambda, p=pitch,z=z1)

    #3.4
    #レンズ
    result2 = lens.convex_lens(result1, wl=Lambda, p=pitch, f1=z2)
    
    #5
    #回折計算
    result3 = cl.angular_spectrum(result2, wl=Lambda, p=pitch, z=z3, out_type="numpy", out_form="amp_float")
    


elif mode=="image":
    for h in range(count):
        print(h)
        for k in range(int(data_train/batch_size)):
            
            #画像データのロード
            X_train1 = opt_util.input_img(folder_path="./1024",batch_size=batch_size, num_per_file=data_train, counter1=h, counter2=k)

            #2
            #回折計算
            result1 = cl.angular_spectrum(X_train1, wl=Lambda, p=pitch,z=z1)
            
            #3.4
            #レンズ
            result2 = lens.convex_lens(result1, wl=Lambda, p=pitch, f1=z2)

            #5
            #回折計算
            result3 = cl.angular_spectrum(result1, wl=Lambda, p=pitch, z=-z1, out_type="numpy", out_form="amp_float")

else :
    pass

0


In [18]:
opt_util.output_img(result3, preview="off")